# Supervised CARE Training
Here we will train a plain N2V network on single noisy images.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from tifffile import imread

from pn2v.unet import UNet
from pn2v import utils
from pn2v import training

# See if we can use a GPU
device=utils.getDevice()

CUDA available? False


### Load Data

In [ ]:
path='data/Convallaria_diaphragm/'

# Load the training data
data=imread(path+'20190520_tl_25um_50msec_05pc_488_130EM_Conv.tif')
dataGT = np.mean(data,axis=0)[np.newaxis,...,np.newaxis]

data=data[...,np.newaxis]

dataGT = np.repeat(dataGT, 100, axis=0)
print(data.shape,dataGT.shape)
data = np.concatenate((data,dataGT),axis=-1)
print(data.shape,dataGT.shape)




plt.imshow(data[0,512:,512:,0])
plt.show()
plt.imshow(data[0,:512,512:,0])
plt.show()
plt.imshow(data[0,512:,:512,0])
plt.show()

plt.imshow(data[0,:512,:512,0])
plt.show()

# We now crop away the data that has is to be used for testing
data=np.concatenate( (data[:,512:,512:,:], data[:,:512,512:,:], data[:,512:,:512,:])  )


### Create the Network and Train it
This can take a while.

In [ ]:
# The N2V network requires only a single output unit per pixel
net = UNet(1, depth=3)

# Split training and validation data.
my_train_data=data[:-5].copy()
my_val_data=data[-5:].copy()

# Start training.
trainHist, valHist = training.trainNetwork(net=net, trainData=my_train_data, valData=my_val_data,
                                           postfix='conv_CARE', directory=path, noiseModel=None,
                                           device=device, numOfEpochs= 200, stepsPerEpoch=5, 
                                           virtualBatchSize=20, batchSize=1, learningRate=1e-3, supervised=True)

In [ ]:
# Let's look at the training and validation loss
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(valHist, label='validation loss')
plt.plot(trainHist, label='training loss')
plt.legend()
plt.show()